In [1]:
import numpy as np
import tqdm 
import os
import sys
import matplotlib.pyplot as plt
from matplotlib import colors 
import matplotlib as mpl
from matplotlib.cm import ScalarMappable

sys.path.append('../pipeline_scripts/')
from pipeline_main import pipeline
from pipeline_streamers import phi_average 


# Set som plotting standards:
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 20}
mpl.rc('font', **font)

In [2]:
datai = pipeline(snap = 343, run = 'sink122core02', sink_id=13) 
data = pipeline(snap = 400, run = 'sink122core02', sink_id=122)
data.recalc_L(verbose = 0); data.calc_cyl(); data.calc_trans_xyz(verbose = 0)

Initialising patch data
Assigning relative cartesian velocities and coordinates to all cells
Assigning masses to all cells
Calculating adiabatic index γ and pressure (polytropic) for all cells


100%|██████████| 4165/4165 [00:05<00:00, 761.76it/s]


Initialising patch data
Assigning relative cartesian velocities and coordinates to all cells
Assigning masses to all cells
Calculating adiabatic index γ and pressure (polytropic) for all cells


100%|██████████| 4165/4165 [00:00<00:00, 4956.65it/s]


In [3]:
data.sink_evolution()

Loading all snapshots - this might take awhile


100%|██████████| 59/59 [01:12<00:00,  1.23s/it]


In [4]:
snapshots = np.asarray([key for key in data.snaps.keys()])

In [6]:
orig_directory = os.getcwd()
os.chdir('graphics')
folder = "phiaverage_ρevolution"
try:
    os.mkdir(folder)
except:
    pass
os.chdir('../')

for sn in tqdm.tqdm(snapshots):
    data_loop = pipeline(snap = sn, run = 'sink122core02', sink_id=122, verbose=0, loading_bar=False)
    data_loop.recalc_L(verbose = 0); data_loop.calc_cyl(); data_loop.calc_trans_xyz(verbose = 0)

    R, z, val_P,  rr_v, zz_v, vr, vz, arrow_length = data_loop.phiaverage(ivs = None, radius=80, NR = 120, quiver_dens=0.4, get_quiver=True, plot = False)
    
    β_phiaverage = val_P * data.cgs_pressure / (val_B * data.sn.scaling.b**2 / (8 * np.pi))

    xx, yy = np.meshgrid(R * data.au_length, z * data.au_length)
    quiver_dens = 0.45

    fig, axs = plt.subplots(figsize = (20,8))

    log_vmin = -4; log_vmax = 4
    cs = axs.contourf(R * data.au_length, z * data.au_length, np.log10(β_phiaverage.T), origin = 'lower', levels = 200, cmap = 'RdBu', vmin = log_vmin, vmax = log_vmax)

    cbar = fig.colorbar(ScalarMappable(norm=cs.norm, cmap=cs.cmap), ticks = range(log_vmin, log_vmax + 1, 1), ax = axs, fraction = 0.1, pad = 0.06, location = 'top');


    quiver = axs.quiver(rr_v * data.au_length, zz_v*data.au_length, vr , vz, np.log10(arrow_length), 
                                cmap=mpl.cm.cividis_r, headwidth = 2.5, headaxislength = 2.3, headlength = 2.3, pivot = 'mid', scale = 100 / (0.6 / quiver_dens));
    cbar_vel = fig.colorbar(quiver, pad = 0.005)
    cbar_vel.set_label('$\log_{10}(V)$ [cm/s]')

    ct = axs.contour(xx, yy, np.log10(β_phiaverage.T), [np.log10(1)], colors = ['black'], alpha = 0.6, ls = '--')

    contour_labels = ['β = 1']
    for i in range(len(contour_labels)):
        ct.collections[i].set_label(contour_labels[i])

    cbar.set_label('log10(β)\n[Unitless]', labelpad = -60, x = -0.08, rotation = 0, fontsize = 18)
    axs.set(ylabel = 'Height over midplane [AU]', xlabel = 'Distance from star  [AU]');
    axs.set(title=f'T = {(data_loop.time - datai.time)*1e-3:2.1f} kyr')
    plt.legend()
    os.chdir('graphics')
    plt.savefig(folder + f'/snap_{sn:3.0f}.png', facecolor='w')
    os.chdir('../')
    plt.close(fig)

  0%|          | 0/59 [00:00<?, ?it/s]/tmp/ipykernel_1364817/1018684282.py:34: UserWarning: The following kwargs were not used by contour: 'ls'
  ct = axs.contour(xx, yy, np.log10(β_phiaverage.T), [np.log10(1)], colors = ['black'], alpha = 0.6, ls = '--')
100%|██████████| 59/59 [31:19<00:00, 31.86s/it]


In [5]:
2e-5 * 3000

0.060000000000000005